### Create figure 3

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[6], ext_colors[4]]

mypalette = sns.color_palette(ext_colors)

shade_gray = 0.8
shade_red = 0.8
shade_alpha = 0.5

mypalette

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import os
# import MultipleLocator
from matplotlib.ticker import MultipleLocator

host_machines = [
  'safari-fpga37',
  'safari-fpga55',
  'safari-fpga56',
  'safari-fpga57',
  'safari-fpga58',
  'safari-fpga59'
]

rows_tested_for_bitflips = 3072

dfs = []

# for each h as host machine, read file 'h-temperature.txt' into a single dataframe
# there is only one column, temperature in Celsius
# also add a new column to the DF for each temperature you read from the file
# these are readings 5 seconds apart, so we add a timestamp column that starts at 0 and increments by 5
# for each row
for host_machine in host_machines:
  df = pd.read_csv(f'fast_forward_data/{host_machine}-temperature.txt', header=None, names=['temperature'])
  df['machine'] = host_machine
  df['timestamp'] = df.index * 5
  dfs.append(df)

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# new column minutes (as integer)
df['minutes'] = df['timestamp'] // 60
df['hours'] = df['minutes'] // 60

# remove entries after 24 hours
df = df[df['hours'] < 24]

# group by minutes and get mean
# df = df.groupby(['machine', 'minutes']).mean().reset_index()

# rename machine using the following map
machine_map = {
  'safari-fpga37': 'Chip 0',
  'safari-fpga55': 'Chip 1',
  'safari-fpga56': 'Chip 2',
  'safari-fpga57': 'Chip 3',
  'safari-fpga58': 'Chip 4',
  'safari-fpga59': 'Chip 5'
}
df['machine'] = df['machine'].map(machine_map)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

df#  chips of interest
chips_of_interest = ['Chip 0', 'Chip 1', 'Chip 2', 'Chip 3', 'Chip 4', 'Chip 5']

# visualize how temperature changes for one chip using scatterplot
fig, axes = plt.subplots(figsize=(12, 2), nrows=1, ncols=len(chips_of_interest))

# more space between axes

# remove temperature above 84 from Chip 0 only
df = df[~((df['chip'] == 'Chip 0') & (df['temperature'] > 83))]
df = df[~((df['chip'] == 'Chip 0') & (df['temperature'] < 81))]

i = 0
#for chip in ['Chip 0', 'Chip 1', 'Chip 2', 'Chip 3', 'Chip 4', 'Chip 5']:
for chip, ax in zip(chips_of_interest, axes):
    # scatterplot circle edge color is mypalette[0]
    # scatterplot circle fill color is mypalette[0]
    # scatterplot circle edge width is 0.5
    # scatterplot circle size is 10
    sns.scatterplot(data=df[df['chip'] == chip], x='minutes', y='temperature', ax=ax, color=mypalette[0], edgecolor=mypalette[0], linewidth=0, s=10)

    ax.set_ylabel('')
    if chip == 'Chip 0':
        ax.set_ylabel('Temperature ($\degree{}$C)', fontsize=12)
    ax.set_xlabel('Hours', fontsize=12)
    # rename x axis tick labels by dividing them by 360
    # ax.set_xticklabels([int(x) // 3600 for x in ax.get_xticks()])

    # draw a tick every 6*3600 timestamp
    # ax.xaxis.set_major_locator(MultipleLocator(6*3600))
    # ax.set_xticklabels([int(x) // 3600 for x in ax.get_xticks()])

    ax.set_yticks(np.arange(df[df['chip'] == chip]['temperature'].min(), df[df['chip'] == chip]['temperature'].max() + 1, 1))

    ax.xaxis.set_major_locator(MultipleLocator(240))
    ax.set_xticklabels([int(x) // 60 for x in ax.get_xticks()])

    # have 5 y axis ticks starting from minimum temperature 
    # ax.set_yticks(np.arange(df[df['chip'] == chip]['temperature'].min(), df[df['chip'] == chip]['temperature'].min() + 6, 1))

    ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    ax.set_title(chip, fontsize=12)

# add title to both subplots
# axes[0].set_title('Chip 2', fontsize=12)
# axes[1].set_title('Chip 5', fontsize=12)

# tight layout
fig.tight_layout()

fig.savefig('figure3.pdf', bbox_inches='tight')
fig.savefig('figure3.png', bbox_inches='tight', dpi=1000)


### Create figures 4 and 6

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[2], ext_colors[4], ext_colors[6], ext_colors[7], ext_colors[5], ext_colors[3], ext_colors[1]]

mypalette = sns.color_palette(ext_colors)

mypalette

# read all csv files in this folder and concatenate them into one dataframe
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

host_machines = [
  'safari-fpga37',
  'safari-fpga55',
  'safari-fpga56',
  'safari-fpga57',
  'safari-fpga58',
  'safari-fpga59'
]

rows_tested_for_bitflips = 3072

dfs = []

# Loop over all CSV files in the directory
for host_machine in host_machines:
  dir_path = './' + 'fast_forward_data/' + host_machine
  for filename in os.listdir(dir_path):
    if filename.startswith('rows_only_rowhammer') and 'tAggOn' not in filename:
      tokens = filename.split('_')
      ch_num = int(tokens[4])
      pc_num = int(tokens[6])
      bank_num = int(tokens[8])
      pattern = tokens[10].split('.')[0]
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename)
      # add the channel, pc, bank and pattern numbers as columns
      df['CH'] = ch_num
      df['PC'] = pc_num
      df['bank'] = bank_num
      df['pattern'] = pattern
      df['machine'] = host_machine
      # append the dataframe to the list
      dfs.append(df)

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# map the channels by decrementing them by 8
df['CH'] = df['CH'].map(lambda x: x - 8)

# map the rows in the following ranges to the following ranges
# [0-1023] -> [0-1023]
# [1024-2047] -> [7168-8191]
# [2048-3071] -> [13310-14333]

def map_row(row):
  if row < 1024:
    return row
  elif row < 2048:
    return row + 6144
  else:
    return row + 11262

common_rows = [
  range(0, 1024, 1),
  range(7168, 8192, 1),
  range(13310, 14334, 1)
]

common_banks = [
  0, 1, 2
]

# group by machine, channel, pc, bank, pattern
# if the group has 3072 rows, then apply map row
# otherwise, leave the row as is
df['row'] = df.groupby(['machine', 'CH', 'PC', 'bank', 'pattern'])['row'].transform(lambda x: x.map(map_row) if len(x) == rows_tested_for_bitflips else x)

# rename machine using the following map
machine_map = {
  'safari-fpga37': 'Chip 0',
  'safari-fpga55': 'Chip 1',
  'safari-fpga56': 'Chip 2',
  'safari-fpga57': 'Chip 3',
  'safari-fpga58': 'Chip 4',
  'safari-fpga59': 'Chip 5'
}
df['machine'] = df['machine'].map(machine_map)

pattern_map = {
  '0': '00',
  '1': 'FF',
  '2': '55',
  '3': 'AA'
}

# modify pattern column only if the pattern is not one of the values in the map
df['pattern'] = df['pattern'].map(lambda x: pattern_map[x] if x in pattern_map else x)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

# rename row_errors to bitflips
df.rename(columns={'row_errors': 'bitflips'}, inplace=True)

df['bitflips'] = df['bitflips'] / 5

# create bit error rate column 
# ber = bitflips / row size (1024*8)
df['ber'] = df['bitflips'] / (1024*8) * 100

# filter out uncommon rows
df_common_rows = df[df['row'].isin(common_rows[0]) | df['row'].isin(common_rows[1]) | df['row'].isin(common_rows[2])].copy()
df_common_banks_rows = df_common_rows[df_common_rows['bank'].isin(common_banks)].copy()

# a new data point for each row is created for a new worst case data pattern
# this is the data pattern that maximizes ber for a row in a given channel, pc, bank, chip, with different patterns

# group by chip, channel, pc, bank, row
# for each group, find the pattern that maximizes ber
# create a new dataframe with the following columns: chip, channel, pc, bank, row, pattern, ber
df_worst_case = df_common_banks_rows.groupby(['chip', 'CH', 'PC', 'bank', 'row'])[['pattern', 'ber']].max().reset_index()

df_worst_case.reset_index(drop=True, inplace=True)

# rename pattern values with "WCDP"
df_worst_case['pattern'] = df_worst_case['pattern'].map(lambda x: 'WCDP')

# add df_worst_case to df
df_common_banks_rows = pd.concat([df_common_banks_rows, df_worst_case], ignore_index=True)

df_common_banks_rows


ber_chip_ch = df_common_banks_rows.copy()

# flatten the multi-index
ber_chip_ch.reset_index(inplace=True)

# if ber smaller than 0.1%, remove the row
ber_chip_ch = ber_chip_ch[ber_chip_ch['ber'] > 0.1]

# select only chip, channel and bitflips
ber_chip_ch = ber_chip_ch[['chip', 'CH', 'pattern', 'ber']]
ber_chip_ch

# order for the pattern column is:
pattern_order = ['FF', '00', 'AA', '55', 'WCDP']
# apply this order
ber_chip_ch['pattern'] = pd.Categorical(ber_chip_ch['pattern'], pattern_order)

# sort by pattern, chip, channel
ber_chip_ch.sort_values(by=['pattern', 'chip', 'CH'], inplace=True)


# create a subplot for each chip
fig, axes = plt.subplots(nrows=len(ber_chip_ch['chip'].unique()), figsize=(8.1, 2.0*len(ber_chip_ch['chip'].unique())))

data_patterns = {
  0: "Rowstripe0",
  1: "Rowstripe1",
  2: "Checkered0",
  3: "Checkered1",
  4: "WCDP"
}

# create a bar plot for each chip
for chip, ax in zip(ber_chip_ch['chip'].unique(), axes):
  sns.boxplot(x='pattern', y='ber', hue='CH', data=ber_chip_ch[ber_chip_ch['chip'] == chip], ax=ax, palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"3"})
  # add text saying Chip X in a box
  ax.text(0.1, 0.85, chip, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
  ax.get_legend().remove()
  # if chip != 'Chip 5':
  ax.set_xlabel('')
  ax.set_xticklabels([])
  ax.set_xticks([])
  if chip == 'Chip 0':
    # prepend 'CH' to legend labels
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['CH' + label for label in labels], loc='upper center', title='Channel', ncol=8, bbox_to_anchor=(0.5, 1.5), fancybox=True, shadow=True, columnspacing=0.5)
  if chip == 'Chip 5':
    ax.set_xticks(range(5))
    ax.set_xticklabels([data_patterns[i] for i in range(5)])
    ax.set_xlabel('Data pattern')
  ax.set_ylabel('')
  # ax.set_title('Bank {}'.format(b))
  # put horizontal grid lines in the background
  ax.set_axisbelow(True)
  shade_gray = 0.8
  ax.axvspan(3.5, 4.5, facecolor=(shade_gray,shade_gray,shade_gray), alpha=0.2)
  ax.axvline(3.5, color='black', linestyle='dashed')
  ax.set_xlim(-0.5, 4.5)
  ax.yaxis.grid(color='gray', linestyle='dashed')
  ax.set_ylim(0, 3.2)
  ax.set_yticks([0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2])
  ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
  # remove legend
  # ax.set_yticklabels([str(x*100) for x in ax.get_yticks()])

# add text as y axis label that spans all subplots
fig.text(0.05, 0.5, 'RowHammer Bit Error Rate ($BER$)', va='center', rotation='vertical', fontsize=12)

# set subplots apart
# fig.tight_layout()
fig.subplots_adjust(hspace=0.15)

# save plot
fig.savefig('figure6.pdf', bbox_inches='tight')

# print the average BER across everything for WCDP
print('Max BER across everything for WCDP: {}'.format(ber_chip_ch[ber_chip_ch['pattern'] == 'WCDP']['ber'].max()))
print(ber_chip_ch[ber_chip_ch['pattern'] == 'WCDP']['ber'].max()*8192/100)

# print the average bit error rate for Chip 0 CH7 for WCDP
print('Average BER for Chip 0 CH7 for WCDP: {}'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'WCDP') & (ber_chip_ch['chip'] == 'Chip 0') & (ber_chip_ch['CH'] == 7)]['ber'].mean()))
print('Average BER for Chip 0 CH3 for WCDP: {}'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'WCDP') & (ber_chip_ch['chip'] == 'Chip 0') & (ber_chip_ch['CH'] == 3)]['ber'].mean()))


print('Average BER for Chip 4 CH7 for WCDP: {}'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'WCDP') & (ber_chip_ch['chip'] == 'Chip 4') & (ber_chip_ch['CH'] == 7)]['ber'].mean()))
print('Average BER for Chip 4 CH1 for WCDP: {}'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'WCDP') & (ber_chip_ch['chip'] == 'Chip 4') & (ber_chip_ch['CH'] == 1)]['ber'].mean()))


# print by how much CH7 is worse than CH3
print('CH7 is worse than CH3 by: {}'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'WCDP') & (ber_chip_ch['chip'] == 'Chip 0') & (ber_chip_ch['CH'] == 7)]['ber'].mean() / ber_chip_ch[(ber_chip_ch['pattern'] == 'WCDP') & (ber_chip_ch['chip'] == 'Chip 0') & (ber_chip_ch['CH'] == 3)]['ber'].mean()))

print('Maximum observed BER in CH7 in Chip 0 for RowStripe 1: {}'.format(ber_chip_ch[(ber_chip_ch['pattern'] == '00') & (ber_chip_ch['chip'] == 'Chip 0') & (ber_chip_ch['CH'] == 7)]['ber'].max()))
print('Maximum observed BER in CH7 in Chip 0 for Checkered 0: {}'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 0') & (ber_chip_ch['CH'] == 7)]['ber'].max()))

# print the average BER for each chip
for chip in ber_chip_ch['chip'].unique():
  print('Average BER for Chip {}: {}'.format(chip, ber_chip_ch[(ber_chip_ch['pattern'] == 'WCDP') & (ber_chip_ch['chip'] == chip)]['ber'].mean()))

# seledct chip 4 and checkered 0
ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 4')]

# group by channels, take mean of BER, find min and max of those
# then find the difference between the min and max
ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 4')].groupby('CH')['ber'].mean().max() - ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 4')].groupby('CH')['ber'].mean().min() 


fig = plt.figure(figsize=(8.1, 2.0))

sns.boxplot(x='pattern', y='ber', hue='chip', data=ber_chip_ch, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"3"}, palette=sns.color_palette("Set2", 6))

# get figure's ax
fig = plt.gca()

# move legend above figure
handles, labels = fig.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', title='Chip', ncol=6, bbox_to_anchor=(0.5, 1.4), fancybox=True, shadow=True, columnspacing=0.5)

fig.set_xticks(range(5))
fig.set_xticklabels([data_patterns[i] for i in range(5)])
fig.set_xlabel('Data pattern')
fig.set_ylabel('RowHammer BER (%)')
# ax.set_title('Bank {}'.format(b))
# put horizontal grid lines in the background
fig.set_axisbelow(True)
shade_gray = 0.8
fig.axvspan(3.5, 4.5, facecolor=(shade_gray,shade_gray,shade_gray), alpha=0.2)
fig.axvline(3.5, color='black', linestyle='dashed')
fig.set_xlim(-0.5, 4.5)
fig.yaxis.grid(color='gray', linestyle='dashed')
fig.set_ylim(0, 3.2)
fig.set_yticks([0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2])
fig.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
# remove legend
# ax.set_yticklabels([str(x*100) for x in ax.get_yticks()])

# save figure
plt.savefig('figure4.pdf', bbox_inches='tight')

# Chip 0 and Chip 5 mean BER for Checkered 0
print('Chip 0 max BER for Checkered 0: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 0')]['ber'].max()))
print('Chip 0 mean BER for Checkered 0: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 0')]['ber'].mean()))
print('Chip 5 max BER for Checkered 0: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 5')]['ber'].max()))
print('Chip 5 mean BER for Checkered 0: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') & (ber_chip_ch['chip'] == 'Chip 5')]['ber'].mean()))


print('Mean BER for Checkered 0 and 1: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA') | (ber_chip_ch['pattern'] == '55')]['ber'].mean()))
print('Mean BER for Rowstripe 0 and 1: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'FF') | (ber_chip_ch['pattern'] == '00')]['ber'].mean()))


print('Max BER for a chip for Checkered0: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA')].groupby('chip')['ber'].mean().max()))
print('Min BER for a chip for Checkered0: {:.2f}%'.format(ber_chip_ch[(ber_chip_ch['pattern'] == 'AA')].groupby('chip')['ber'].mean().min()))

### Create figures 5 and 7

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[2], ext_colors[4], ext_colors[6], ext_colors[7], ext_colors[5], ext_colors[3], ext_colors[1]]

mypalette = sns.color_palette(ext_colors)

mypalette
 

# read all csv files in this folder and concatenate them into one dataframe
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

host_machines = [
  # 'safari-fgpa37',
  'safari-fpga55',
  'safari-fpga56',
  'safari-fpga57',
  'safari-fpga58',
  'safari-fpga59'
]

dfs = []

# Loop over all CSV files in the directory
for host_machine in host_machines:
  dir_path = './' + 'fast_forward_data/' +  host_machine
  for filename in os.listdir(dir_path):
    if filename.startswith('hcfirst') and 'tAggOn' not in filename:
      tokens = filename.split('_')
      ch_num = int(tokens[2])
      pc_num = int(tokens[4])
      bank_num = int(tokens[6])
      pattern = tokens[8].split('.')[0]
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename, header=None, names=['hcfirst'])
      # add the channel, pc, bank and pattern numhcfirsts as columns
      df['CH'] = ch_num
      df['PC'] = pc_num
      df['bank'] = bank_num
      df['pattern'] = pattern
      df['machine'] = host_machine
      df['row'] = df.index
      # append the dataframe to the list
      dfs.append(df)


# Loop over all CSV files in the directory
for host_machine in ['safari-fpga37']:
  dir_path = './' + 'fast_forward_data/' + host_machine
  for filename in os.listdir(dir_path):
    if filename.startswith('hcfirst') and 'tAggOn' not in filename:
      tokens = filename.split('_')
      ch_num = int(tokens[2])
      pc_num = 0
      bank_num = int(tokens[4])
      pattern = tokens[6].split('.')[0]
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename, header=None, names=['hcfirst'])
      # add the channel, pc, bank and pattern numhcfirsts as columns
      df['CH'] = ch_num
      df['PC'] = pc_num
      df['bank'] = bank_num
      df['pattern'] = pattern
      df['machine'] = host_machine
      df['row'] = df.index

      # remove entries with row in the following ranges
      # [1024-3095], [4120-6167], [7168, 9215]
      df = df[~df['row'].between(1024, 3095)]
      df = df[~df['row'].between(4120, 6167)]
      df = df[~df['row'].between(7168, 9215)]
      # append the dataframe to the list
      dfs.append(df)

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# map the channels by decrementing them by 8
df['CH'] = df['CH'].map(lambda x: x - 8)

# map the rows in the following ranges to the following ranges
# [0-255] -> [0-255]
# [256-511] -> [7936-8191]
# [2048-3071] -> [15614-15869]

def map_row(row):
  if row < 1024:
    return row
  elif row < 2048:
    return row + 7168
  else:
    return row + 13310

# apply map_row to the row column
df['row'] = df['row'].map(map_row)

# rename machine using the following map
machine_map = {
  'safari-fpga37': 'Chip 0',
  'safari-fpga55': 'Chip 1',
  'safari-fpga56': 'Chip 2',
  'safari-fpga57': 'Chip 3',
  'safari-fpga58': 'Chip 4',
  'safari-fpga59': 'Chip 5'
}
df['machine'] = df['machine'].map(machine_map)

pattern_map = {
  '0': '00',
  '1': 'FF',
  '2': '55',
  '3': 'AA'
}

# modify pattern column only if the pattern is not one of the values in the map
df['pattern'] = df['pattern'].map(lambda x: pattern_map[x] if x in pattern_map else x)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

# a new data point for each row is created for a new worst case data pattern
# this is the data pattern that maximizes hcfirst for a row in a given channel, pc, bank, chip, with different patterns

# remove rows with hcfirst >= 256000
df = df[df['hcfirst'] < 256000]

# a row should have an entry for each pattern
# if a row does not have an entry for a pattern, then remove it completely
# remove rows that do not have all patterns
df = df.groupby(['chip', 'CH', 'PC', 'row', 'bank']).filter(lambda x: len(x['pattern'].unique()) == 4)

# group by chip, channel, pc, bank, row
# for each group, find the pattern that maximizes hcfirst
# create a new dataframe with the following columns: chip, channel, pc, bank, row, pattern, ber
df_worst_case = df.groupby(['chip', 'CH', 'PC', 'bank', 'row'])[['pattern', 'hcfirst']].min().reset_index()

df_worst_case.reset_index(drop=True, inplace=True)


# rename pattern values with "WCDP"
df_worst_case['pattern'] = df_worst_case['pattern'].map(lambda x: 'WCDP')

# add df_worst_case to df
df = pd.concat([df, df_worst_case], ignore_index=True)


hcf_chip_ch = df.copy()

# flatten the multi-index
hcf_chip_ch.reset_index(inplace=True)

# select only chip, channel and bitflips
hcf_chip_ch = hcf_chip_ch[['chip', 'CH', 'pattern', 'hcfirst', 'row', 'bank', 'PC']]

# order chips
chip_order = ['Chip 0', 'Chip 1', 'Chip 2', 'Chip 3', 'Chip 4', 'Chip 5']
# apply this order
hcf_chip_ch['chip'] = pd.Categorical(hcf_chip_ch['chip'], chip_order)

# order for the pattern column is:
pattern_order = ['FF', '00', 'AA', '55', 'WCDP']
# apply this order
hcf_chip_ch['pattern'] = pd.Categorical(hcf_chip_ch['pattern'], pattern_order)

# sort by pattern, chip, channel
hcf_chip_ch.sort_values(by=['pattern', 'chip', 'CH'], inplace=True)

# create a subplot for each chip
fig, axes = plt.subplots(nrows=len(hcf_chip_ch['chip'].unique()), figsize=(8.1, 2.0*len(hcf_chip_ch['chip'].unique())))

data_patterns = {
  0: "Rowstripe0",
  1: "Rowstripe1",
  2: "Checkered0",
  3: "Checkered1",
  4: "WCDP"
}

# create a bar plot for each chip
for chip, ax in zip(hcf_chip_ch['chip'].unique(), axes):
  sns.boxplot(x='pattern', y='hcfirst', data=hcf_chip_ch[hcf_chip_ch['chip'] == chip], ax=ax, showfliers=False, hue="CH", palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"3"})
  ax.text(0.1, 0.85, chip, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
  ax.get_legend().remove()
  ax.set_xlabel('')
  ax.set_xticklabels([])
  ax.set_xticks([])

  if chip == 'Chip 0':
    # prepend 'CH' to legend labels
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['CH' + label for label in labels], loc='upper center', title='Channel', ncol=8, bbox_to_anchor=(0.5, 1.5), fancybox=True, shadow=True, columnspacing=0.5)
  if chip == 'Chip 5':
    ax.set_xticks(range(5))
    ax.set_xticklabels([data_patterns[i] for i in range(5)])
    ax.set_xlabel('Data pattern')

  ax.set_yticks([x * 40000 for x in range(7)])
  ax.set_yticklabels([str(int(int(t) / 1000))+"K" if t > 0 else "0" for t in ax.get_yticks()])
  ax.set_ylabel('')

  # Put grid lines in the background
  ax.set_axisbelow(True)
  # # put a shade on the background of worst case
  shade_gray = 0.8
  ax.axvspan(3.5, 4.5, facecolor=(shade_gray,shade_gray,shade_gray), alpha=0.2)
  ax.axvline(3.5, color='black', linestyle='dashed')
  ax.set_xlim(-0.5, 4.5)
  ax.grid(color='gray', linestyle='dashed')

  # Put grid lines only on y-axis
  ax.xaxis.grid(False)

fig.subplots_adjust(hspace=0.15)
    

# add text as y axis label that spans all subplots
fig.text(0.04, 0.5, 'Minimum Hammer Count to Induce the First Bitflip ($HC_{first}$)', va='center', rotation='vertical', fontsize=12)

# save plot
fig.savefig('figure7.pdf', bbox_inches='tight')

# minimum hcfirst across everything
min_hcfirst = df['hcfirst'].min()
print("Minimum hcfirst across everything: {}".format(min_hcfirst))

# mean hcfirst for Rowstripe0 in chip 1 channel 0
mean_hcfirst = df[(df['chip'] == 'Chip 1') & (df['CH'] == 0) & (df['pattern'] == 'FF')]['hcfirst'].median()
print("Mean hcfirst for Rowstripe0 in chip 1 channel 0: {}".format(mean_hcfirst))

# mean hcfirst for Rowstripe1 in chip 1 channel 0
mean_hcfirst = df[(df['chip'] == 'Chip 1') & (df['CH'] == 0) & (df['pattern'] == '00')]['hcfirst'].median()
print("Mean hcfirst for Rowstripe1 in chip 1 channel 0: {}".format(mean_hcfirst))

fig = plt.figure(figsize=(8.1, 2.0))

sns.boxplot(x='pattern', y='hcfirst', hue='chip', data=hcf_chip_ch, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"3"}, palette=sns.color_palette("Set2", 6))

# get figure's ax
fig = plt.gca()

# move legend above figure
handles, labels = fig.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', title='Chip', ncol=6, bbox_to_anchor=(0.5, 1.4), fancybox=True, shadow=True, columnspacing=0.5)

fig.set_xticks(range(5))
fig.set_xticklabels([data_patterns[i] for i in range(5)])
fig.set_xlabel('Data pattern')
fig.set_ylabel('Minimum Hammer Count\nto Induce the First Bitflip')
# ax.set_title('Bank {}'.format(b))
# put horizontal grid lines in the background
fig.set_axisbelow(True)
shade_gray = 0.8
fig.axvspan(3.5, 4.5, facecolor=(shade_gray,shade_gray,shade_gray), alpha=0.2)
fig.axvline(3.5, color='black', linestyle='dashed')
fig.set_xlim(-0.5, 4.5)
fig.yaxis.grid(color='gray', linestyle='dashed')
fig.set_yticks([x * 40000 for x in range(7)])
fig.set_yticklabels([str(int(int(t) / 1000))+"K" if t > 0 else "0" for t in fig.get_yticks()])

# save plot
plt.savefig('figure5.pdf', bbox_inches='tight')


print('Minimum hcfirst across everything: {:.0f}'.format(hcf_chip_ch['hcfirst'].min()))
print('This hcfirst comes from chip {} and pattern {}'.format(hcf_chip_ch[hcf_chip_ch['hcfirst'] == hcf_chip_ch['hcfirst'].min()]['chip'].values[0], hcf_chip_ch[hcf_chip_ch['hcfirst'] == hcf_chip_ch['hcfirst'].min()]['pattern'].values[0]))
print('Minimum hcfirst across everything takes {:.0f} microseconds'.format(hcf_chip_ch['hcfirst'].min()*45.0*2/1000)) # 45 taken from oconnor thesis

# min HCfirst in each chip (for all patterns)
print('Chip 0 min HCfirst for all patterns: {:.0f}'.format(hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 0']['hcfirst'].min()))
print('Chip 1 min HCfirst for all patterns: {:.0f}'.format(hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 1']['hcfirst'].min()))
print('Chip 2 min HCfirst for all patterns: {:.0f}'.format(hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 2']['hcfirst'].min()))
print('Chip 3 min HCfirst for all patterns: {:.0f}'.format(hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 3']['hcfirst'].min()))
print('Chip 4 min HCfirst for all patterns: {:.0f}'.format(hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 4']['hcfirst'].min()))
print('Chip 5 min HCfirst for all patterns: {:.0f}'.format(hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 5']['hcfirst'].min()))

# the row with the min HCfirst for each chip
chip4_df = hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 4']
print('This hcfirst comes from row {} in Chip 4'.format(chip4_df[chip4_df['hcfirst'] == chip4_df['hcfirst'].min()][['row', 'bank', 'CH', 'PC']].values))

# and in chip 2
chip2_df = hcf_chip_ch[hcf_chip_ch['chip'] == 'Chip 2']
print('This hcfirst comes from row {} in Chip 2'.format(chip2_df[chip2_df['hcfirst'] == chip2_df['hcfirst'].min()][['row', 'bank', 'CH', 'PC']].values))

# Chip 2 and Chip 5 mean hcfirst for Rowstripe0
print('Chip 2 max HCfirst for Checkered 0: {:.0f}'.format(hcf_chip_ch[(hcf_chip_ch['pattern'] == 'FF') & (hcf_chip_ch['chip'] == 'Chip 2')]['hcfirst'].mean()))
print('Chip 5 max HCfirst for Checkered 0: {:.0f}'.format(hcf_chip_ch[(hcf_chip_ch['pattern'] == 'FF') & (hcf_chip_ch['chip'] == 'Chip 5')]['hcfirst'].mean()))
print(104548/94540)
#fig.set_ylim(0, 3.2)
#fig.set_yticks([0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2])
# fig.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
# remove legend
# ax.set_yticklabels([str(x*100) for x in ax.get_yticks()])

# group by chip, CH. find minimum hcfirst. then add those to the dataframe

minhcfs = hcf_chip_ch.groupby(['chip','CH'])['hcfirst'].min().reset_index().groupby('chip')['hcfirst'].min().reset_index()
maxhcfs = hcf_chip_ch.groupby(['chip','CH'])['hcfirst'].min().reset_index().groupby('chip')['hcfirst'].max().reset_index()

### Create figure 8

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[6], ext_colors[4]]

mypalette = sns.color_palette(ext_colors)

shade_gray = 0.8
shade_red = 0.8
shade_alpha = 0.5

mypalette

# read all csv files in this folder and concatenate them into one dataframe
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

host_machines = [
  'safari-fpga37',
  'safari-fpga55',
  'safari-fpga56',
  'safari-fpga57',
  'safari-fpga58',
  'safari-fpga59'
]

rows_tested_for_bitflips = 3072

dfs = []

# Loop over all CSV files in the directory
for host_machine in host_machines:
  dir_path = './' + 'fast_forward_data/' + host_machine
  for filename in os.listdir(dir_path):
    if filename.startswith('rows_only_rowhammer') and 'tAggOn' not in filename:
      tokens = filename.split('_')
      ch_num = int(tokens[4])
      if ch_num > 10:
        continue
      pc_num = int(tokens[6])
      if pc_num > 0:
        continue
      bank_num = int(tokens[8])
      if bank_num > 0:
        continue
      pattern = tokens[10].split('.')[0]
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename)
      # add the channel, pc, bank and pattern numbers as columns
      df['CH'] = ch_num
      df['PC'] = pc_num
      df['bank'] = bank_num
      df['pattern'] = pattern
      df['machine'] = host_machine
      # append the dataframe to the list
      dfs.append(df)

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# map the channels by decrementing them by 8
df['CH'] = df['CH'].map(lambda x: x - 8)

# rename machine using the following map
machine_map = {
  'safari-fpga37': 'Chip 0',
  'safari-fpga55': 'Chip 1',
  'safari-fpga56': 'Chip 2',
  'safari-fpga57': 'Chip 3',
  'safari-fpga58': 'Chip 4',
  'safari-fpga59': 'Chip 5'
}
df['machine'] = df['machine'].map(machine_map)

pattern_map = {
  '0': '00',
  '1': 'FF',
  '2': '55',
  '3': 'AA'
}

# modify pattern column only if the pattern is not one of the values in the map
df['pattern'] = df['pattern'].map(lambda x: pattern_map[x] if x in pattern_map else x)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

# rename row_errors to bitflips
df.rename(columns={'row_errors': 'bitflips'}, inplace=True)

df['bitflips'] = df['bitflips'] / 5

# create bit error rate column 
# ber = bitflips / row size (1024*8)
df['ber'] = df['bitflips'] / (1024*8) * 100

# group by chip, channel, pc, bank, row
# for each group, find the pattern that maximizes ber
# create a new dataframe with the following columns: chip, channel, pc, bank, row, pattern, ber
df_worst_case = df.groupby(['chip', 'CH', 'PC', 'bank', 'row'])[['pattern', 'ber']].max().reset_index()

df_worst_case.reset_index(drop=True, inplace=True)

# rename pattern values with "WCDP"
df_worst_case['pattern'] = df_worst_case['pattern'].map(lambda x: 'WCDP')

# add df_worst_case to df
df = pd.concat([df, df_worst_case], ignore_index=True)

df

FAST = True # create the plot faster
PATTERN = 'WCDP' # use this data pattern
BANK = 0
PC = 0

sample_size = 32

# find row'ids where ber is 0 for chip 0
df_zero_ber = df[(df['chip'] == 'Chip 0') & (df['ber'] <= 0.1) & (df['pattern'] == PATTERN) & (df['bank'] == BANK) & (df['PC'] == PC)]
sorted_boundaries = sorted(df_zero_ber['row'].unique())


# pick every other row id from the sorted list
# this will be used to draw vertical lines on the plot
boundaries = [sorted_boundaries[i] for i in range(len(sorted_boundaries)) if i % 2 == 0]
boundaries.append(0)

sa_pattern_df = df.copy()

fig, axes = plt.subplots(nrows=len(sa_pattern_df['chip'].unique()),figsize=(8.1,1.35*len(sa_pattern_df['chip'].unique())))

# reduce the number of rows to plot by taking the average of 8 row's ber
sa_pattern_df['row'] = sa_pattern_df['row'] // sample_size

if FAST:
  sa_pattern_df = sa_pattern_df.groupby(['chip', 'CH', 'PC', 'bank', 'row', 'pattern'])['ber'].mean().reset_index()

for chip, ax in zip(sa_pattern_df['chip'].unique(), axes):
  if FAST:
    sns.lineplot(x="row", y="ber", hue="CH", data=sa_pattern_df[(sa_pattern_df['chip'] == chip) & (sa_pattern_df['pattern'] == PATTERN) & (sa_pattern_df['bank'] == BANK) & (sa_pattern_df['PC'] == PC)], palette=mypalette, ax = ax, ci=None)
  else:
    sns.lineplot(x="row", y="ber", hue="CH", data=sa_pattern_df[(sa_pattern_df['chip'] == chip) & (sa_pattern_df['pattern'] == PATTERN) & (sa_pattern_df['bank'] == BANK) & (sa_pattern_df['PC'] == PC)], palette=mypalette, ax = ax)
  ax.text(0.037, 0.5, chip, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12, rotation=90)
  ax.get_legend().remove()
  ax.set_xlabel('')
  ax.set_xticklabels([])
  ax.set_xticks([])

  if chip == 'Chip 0':
    # prepend 'CH' to legend labels
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['CH' + label for label in labels], loc='upper center', title='Channel', ncol=8, bbox_to_anchor=(0.5, 1.7), fancybox=True, shadow=True, columnspacing=0.5)
  if chip == 'Chip 5':
    ax.set_xticks([-1, 16383/sample_size-1])
    ax.set_xticklabels([0, 16383])
    # rotate xtick labels
    # for tick in ax.get_xticklabels():
    #   tick.set_rotation(45)      
    ax.set_xlabel('Row ID')

  #ax.set_ylabel('RowHammer BER (%)')
  ax.set_ylabel('')

  #ax.set_ylim(0.2, 2.2)
  #ax.set_yticks([0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1])
  #ax.set_yticks([0.3, 0.9, 1.5, 2.1])
  # find the largest and smallest y values in the plot
  ymax = max(ax.get_ylim())
  ymin = min(ax.get_ylim())
  # set the y limit to the largest y value
  ax.set_ylim(ymin-0.1, ymax+0.1)
  
  # if the avg of ymin and ymax can be expressed with 1 decimal place, use it
  if (ymin+ymax)/2 == round((ymin+ymax)/2, 1):
    ax.set_yticks([ymin, (ymin+ymax)/2, ymax])
  else:
    # if not, use 2 decimal places
    ax.set_yticks([ymin, round((ymin+ymax)/2, 2), ymax])

  ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

  ax.set_xlim(-40, (16384/sample_size)+8)
  
  # ax.set_xlim(0, 1000)

  ax.grid(color='gray', linestyle='dashed')
  # Put grid lines only on y-axis
  ax.xaxis.grid(False)

  # # draw vertical lines
  for boundary in boundaries:
    ax.axvline(x=(boundary/sample_size) - 1, color='black', linestyle='dashed', linewidth=0.5)

  ax.axvspan(boundaries[2]/sample_size, boundaries[3]/sample_size, facecolor=(0,1,0), alpha=0.2)
  if chip == 'Chip 0':
    ax.text((boundaries[2]/sample_size + boundaries[3]/sample_size) / 2, (ymax+ymin)/2 - (ymax-ymin)/2.5, "SA X", horizontalalignment='center', verticalalignment='center', fontsize=8, color='green')
  ax.axvspan(boundaries[6]/sample_size, boundaries[7]/sample_size, facecolor=(0,0,1), alpha=0.2)
  if chip == 'Chip 0':
    ax.text((boundaries[6]/sample_size + boundaries[7]/sample_size) / 2, (ymax+ymin)/2  - (ymax-ymin)/2.5, "SA Y", horizontalalignment='center', verticalalignment='center', fontsize=8, color='blue')

  ax.axvspan(boundaries[8]/sample_size, boundaries[9]/sample_size - 1, facecolor=(1,0.2,0.2), alpha=0.2)
  ax.axvspan(boundaries[18]/sample_size, boundaries[19]/sample_size - 1, facecolor=(1,0.2,0.2), alpha=0.2)


fig.subplots_adjust(hspace=0.15)
fig.text(0.04, 0.5, 'RowHammer Bit Error Rate (%)', va='center', rotation='vertical', fontsize=12)

fig.savefig('figure8.pdf', bbox_inches='tight')

### Create figure 9

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import glob

data_patterns = {
    0: "Rowstripe0",
    1: "Rowstripe1",
    2: "Checkered0",
    3: "Checkered1",
    4: "Random",
    5: "WCDP"
}

channel_labels = {
    11:0, 12:1, 14:2, 9:3, 13:4, 10:5, 8:6, 15:7}

row_size = 1 * 1024 * 8 # 1 KB

banks = [0]
# colors = ["#4c72b0", "#55a868", "#c44e52", "#8172b2", "#ccb974", "#64b5cd"]
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.7 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[2], ext_colors[4], ext_colors[6], ext_colors[7], ext_colors[5], ext_colors[3], ext_colors[1]]

mypalette = sns.color_palette(ext_colors)

mypalette

dfs = []
for f in glob.glob('fast_forward_data/rowhammer_results_PC*/*.txt'):
    if 'bank' not in f:
        continue
    conf_str = f.split('rowhammer_results_PC')[1].split('.txt')[0]
    pc = conf_str.split('/')[0]
    conf_arr = conf_str.split('/')[1].split('_')
    df = pd.read_csv(f, header=None, names=['num_bitflips'])
    df['row'] = df.index
    df['bank'] = int(conf_arr[3])
    df['channel'] = channel_labels[int(conf_arr[1])]
    df['pc'] = int(pc)
    df['data_pattern'] = data_patterns[int(conf_arr[-1])]
    # df['file'] = f
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
df['ber'] = df['num_bitflips'] / row_size *100
df = df[['pc', 'row', 'bank', 'channel', 'data_pattern', 'ber']]
df

pivotidcols = ['channel', 'bank', 'pc', 'row']
pivdf = df.pivot(index=pivotidcols, columns='data_pattern', values='ber').reset_index()
print(pivdf)

# Calculate the minimum of hcf_pattern_i for each row_id
pivdf['max_ber'] = pivdf[['Rowstripe0', 'Rowstripe1', 'Checkered0', 'Checkered1', 'Random']].max(axis=1)

# Find which hcf_patter_i has the minimum value
pivdf['wc_pattern'] = pivdf[['Rowstripe0', 'Rowstripe1', 'Checkered0', 'Checkered1', 'Random']].idxmax(axis=1)

pivotidcols.append('wc_pattern')

# print(pivdf)

df = pivdf.melt(id_vars=pivotidcols, var_name='data_pattern', value_name='ber')

df['berpercent'] = df['ber'] * 100

df = df.sort_values(by=['bank', 'channel', 'data_pattern', 'ber'], ascending=[True, True, True, True])
df

# compare the distributions of ber in different banks across channels and pseudo channels
stats_df = pd.DataFrame()
for (ch, pc, bank), bankdf in df.groupby(['channel', 'pc', 'bank']):
    # print("Channel {}, Pseudo Channel {}, Bank {}".format(ch, pc, bank))
    # print(bankdf['ber'].describe())
    new_row = {'Channel': int(ch), 'Pseudo Channel': int(pc), 'Bank': int(bank), 
        'mean': bankdf['ber'].mean(), 
        'std': bankdf['ber'].std()
    }
    new_row['cov'] = new_row['std'] / new_row['mean']
    stats_df = stats_df.append(new_row, ignore_index=True)
stats_df["Channel"] = stats_df["Channel"].astype(int)
stats_df["Pseudo Channel"] = stats_df["Pseudo Channel"].astype(int)
stats_df["Bank"] = stats_df["Bank"].astype(int)
stats_df

# generate a scatterplot of the mean and cov of ber for each bank, color by channel and style by pseudo channel
fig, ax = plt.subplots(figsize=(8.1, 3))
sns.scatterplot(y="mean", x="cov", data=stats_df, hue="Channel", style="Pseudo Channel", ax=ax, palette=mypalette, markers=['.', '*'], s=200, edgecolor='black', linewidth=0.5)
ax.set_ylabel("Mean of RowHammer BER (%)")
ax.set_xlabel("Coefficient of Variation of the RowHammer BER Distribution")
# separate color and style legends
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles[0:], labels=labels[0:], loc='upper center', bbox_to_anchor=(1.2, 1.1), ncol=1, fancybox=True, shadow=False)
ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed')
plt.tight_layout()
fig.savefig('figure9.pdf', bbox_inches='tight')


minmean = stats_df[stats_df.Channel == 7]["mean"].min()
maxmean = stats_df[stats_df.Channel == 7]["mean"].max()
print("min mean: {}, max mean: {}".format(minmean, maxmean))
print(maxmean - minmean)


    

### Create figures 10 and 11

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[2], ext_colors[4], ext_colors[6], ext_colors[7], ext_colors[5], ext_colors[3], ext_colors[1]]

mypalette = sns.color_palette(ext_colors)

mypalette

# read all csv files in this folder and concatenate them into one dataframe
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

host_machines = [
  'safari-fpga37',
  'safari-fpga55',
  'safari-fpga56',
  'safari-fpga57',
  'safari-fpga58',
  'safari-fpga59'
]

dfs = []

# Loop over all CSV files in the directory
for host_machine in host_machines:
  dir_path = './' + 'fast_forward_data/' + host_machine
  for filename in os.listdir(dir_path):
    if filename.startswith('hcnth') and 'wordlevel' not in filename:
      tokens = filename.split('_')
      ch_num = int(tokens[1].split('-')[1])
      pc_num = int(tokens[2].split('-')[1])
      bank_num = int(tokens[3].split('-')[1])
      pattern = tokens[4].split('.')[0].split('-')[1]
      print(pattern)
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename)
      # add the channel, pc, bank and pattern numhcfirsts as columns
      df['CH'] = ch_num
      df['PC'] = pc_num
      df['bank'] = bank_num
      df['pattern'] = pattern
      df['machine'] = host_machine
      # append the dataframe to the list
      dfs.append(df)

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# map the channels by decrementing them by 8
df['CH'] = df['CH'].map(lambda x: x - 8)

# rename machine using the following map
machine_map = {
  'safari-fpga37': 'Chip 0',
  'safari-fpga55': 'Chip 1',
  'safari-fpga56': 'Chip 2',
  'safari-fpga57': 'Chip 3',
  'safari-fpga58': 'Chip 4',
  'safari-fpga59': 'Chip 5'
}
df['machine'] = df['machine'].map(machine_map)

pattern_map = {
  '0': '00',
  '1': 'FF',
  '2': '55',
  '3': 'AA'
}

# modify pattern column only if the pattern is not one of the values in the map
df['pattern'] = df['pattern'].map(lambda x: pattern_map[x] if x in pattern_map else x)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

# rename hc to hcnth and hcnth to hc
df.rename(columns={'hc': 'hcnth', 'hcnth': 'hc'}, inplace=True)

# remove rows where hc is non-numeric
df = df[df['hc'].apply(lambda x: str(x).isnumeric())]

# convert hc to int and hcnth to int
df['hc'] = df['hc'].astype(int)
df['hcnth'] = df['hcnth'].astype(int)

hcf_chip_ch = df.copy()

# flatten the multi-index
hcf_chip_ch.reset_index(inplace=True)

# select only chip, channel and bitflips
hcf_chip_ch = hcf_chip_ch[['chip', 'CH', 'pattern', 'hc', 'hcnth', 'row']]

# remove rows where CH is not 1 or 6
hcf_chip_ch = hcf_chip_ch[(hcf_chip_ch['CH'] == 1) | (hcf_chip_ch['CH'] == 6)]

# remove "0x" from patterns
hcf_chip_ch['pattern'] = hcf_chip_ch['pattern'].str.replace('0x', '')

# based on row, create 3 categories
# cat1: rows < 64
# cat2: rows < 8256
# cat3: rows < 16384
hcf_chip_ch['row'] = hcf_chip_ch['row'].astype(int)
hcf_chip_ch['row_cat'] = pd.cut(hcf_chip_ch['row'], bins=[0, 64, 8256, 16384], labels=['cat1', 'cat2', 'cat3'])

print(hcf_chip_ch)

# order chips
chip_order = ['Chip 0', 'Chip 1', 'Chip 2', 'Chip 3', 'Chip 4', 'Chip 5']
# apply this order
hcf_chip_ch['chip'] = pd.Categorical(hcf_chip_ch['chip'], chip_order)

# order for the pattern column is:
pattern_order = ['FF', '00', 'AA', '55']
# apply this order
hcf_chip_ch['pattern'] = pd.Categorical(hcf_chip_ch['pattern'], pattern_order)

# sort by pattern, chip, channel
hcf_chip_ch.sort_values(by=['pattern', 'chip', 'CH'], inplace=True)

# create a subplot for each chip
fig = plt.plot(figsize=(8.1, 2.0))

data_patterns = {
  0: "Rowstripe0",
  1: "Rowstripe1",
  2: "Checkered0",
  3: "Checkered1"
}

sns.boxplot(x='hcnth', y='hc', data=hcf_chip_ch[hcf_chip_ch['chip'] == chip], showfliers=False, hue="row_cat", palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"3"})

# we are working on df hcf_chip_ch
# group by chip, CH, pattern, row, row_cat. copy the hc for hcnth = 1 in each group to a new column in the df called hc_baseline

newdf = hcf_chip_ch.groupby(['chip', 'CH', 'pattern', 'row', 'row_cat'], group_keys=True).apply(lambda x: x[x['hcnth'] == 1]['hc']).reset_index().rename(columns={0: 'hc_baseline'})

# merge the newdf with the original df
newdf2 = pd.merge(hcf_chip_ch, newdf, on=['chip', 'CH', 'pattern', 'row', 'row_cat'], how='left')

newdf2.rename(columns={'hc_y':'Baseline HC', 'hc_x':'hc'}, inplace=True)

# remove column level_5
newdf2.drop('level_5', axis=1, inplace=True)

# divide hc by Baseline HC

newdf2['hcratio'] = newdf2['hc'].astype(float)/newdf2['Baseline HC'].astype(float)

fig = plt.figure(figsize=(8.1, 2.0))

# order for the pattern column is:
pattern_order = ['FF', '00', 'AA', '55']
# apply this order
newdf2['pattern'] = pd.Categorical(newdf2['pattern'], pattern_order)

data_patterns = {
  0: "Rowstripe0",
  1: "Rowstripe1",
  2: "Checkered0",
  3: "Checkered1"
}

# plot mean hcratio
sns.boxplot(x='hcnth', y='hcratio', data=newdf2, showfliers=False, hue="pattern", palette=mypalette, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"3"}, whis=(0,100))

# rename y axis label
plt.ylabel('Hammer Count\nNormalized to $HC_{first}$', fontsize=12)

# rename x axis label
# plt.xlabel('Unique bitflip in the same row', fontsize=12)

# place legend top left of plot
plt.legend(loc='upper left', bbox_to_anchor=(0.0, 1.0), ncol=1, fontsize=12)

# rename legend labels with data_patterns
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles, [data_patterns[pattern_order.index(i)] for i in labels], loc='center right', bbox_to_anchor=(1.22, 0.5), ncol=1, fontsize=9, title="Data Pattern")

# draw red dashed line at y=1
plt.axhline(y=1, color='red', linestyle='--')

new_xaxislabels=[
    '$HC_{first}$',
    '$HC_{second}$',
    '$HC_{third}$',
    '$HC_{fourth}$',
    '$HC_{fifth}$',
    '$HC_{sixth}$',
    '$HC_{seventh}$',
    '$HC_{eighth}$',
    '$HC_{ninth}$',
    '$HC_{tenth}$'
]

new_xaxislables2=[
    '$HC_{1^{st}}$',
    '$HC_{2^{nd}}$',
    '$HC_{3^{rd}}$',
    '$HC_{4^{th}}$',
    '$HC_{5^{th}}$',
    '$HC_{6^{th}}$',
    '$HC_{7^{th}}$',
    '$HC_{8^{th}}$',
    '$HC_{9^{th}}$',
    '$HC_{10^{th}}$'
]

# set x-axis tick labels
plt.xticks(np.arange(0, 10, 1), new_xaxislables2, fontsize=10)

# remove x axis label
plt.xlabel('')

# point to the red line with text saying $HC_{first}$
# plt.annotate('$HC_{first}$', xy=(-0.5, 1.0), xytext=(0.12, 1.8), arrowprops=dict(facecolor='red', shrink=0.01), fontsize=12, color='red', ha='center', va='center')
plt.gca().get_xticklabels()[0].set_color('red')

# color first y-axis tick red
plt.gca().get_yticklabels()[1].set_color('red')

# draw major grid lines only for y-axis
plt.grid(axis='y', which='major', color='black', linestyle='dotted', linewidth=0.5)

# save figure as "hcnth_plot.pdf"
plt.savefig('figure10.pdf', bbox_inches='tight')

# smallest HC10th value
print("Smallest HC ratio 10th value: " + str(newdf2[newdf2['hcnth'] == 10]['hcratio'].min()))
print("Highest HC ratio 10th value: " + str(newdf2[newdf2['hcnth'] == 10]['hcratio'].max()))

print("Average HCratio2nd, HCratio4th, HCratio8th and HCratio10th", newdf2[(newdf2['hcnth'].isin([2, 4, 8, 10]))&(newdf2['pattern']=='00')].groupby('hcnth')['hcratio'].mean())

print("Average HCtenth for data patterns", newdf2[(newdf2['hcnth']==10)].groupby('pattern')['hcratio'].mean())

newdf2
import numpy as np
# import stats
from scipy import stats
import matplotlib.ticker as ticker



# select Chip 0, CH 1, pattern FF
# filtered = newdf2[(newdf2['chip'] == 'Chip 0') & (newdf2['CH'] == 1) & (newdf2['pattern'] == 'FF')]
filtered = newdf2.copy()
# sort by Baseline HC
filtered.sort_values(by=['Baseline HC'])

# select hcnth = 1 and 10
f2 = filtered[(filtered['hcnth'] == 1) | (filtered['hcnth'] == 10)]

# sort f2 by hcratio
f2 = f2.sort_values(by=['hcratio'])

# remove hcnth = 1 from f2
f2 = f2[f2['hcnth'] != 1]
fig.clf()
fig,ax = plt.subplots(figsize=(9, 6), ncols=2, nrows=3)

# f2 add column is hc - hcfirst
f2['additional'] = f2['hc'] - f2['Baseline HC']

# for all chips
for r in range(0, 3):
    for c in range(0, 2):
        chip_id = r*2 + c
        # select chip i
        f3 = f2[f2['chip'] == 'Chip ' + str(chip_id)]
               # highlight the region with 90% of the plotted population
        # get the 90% quantile of hcratio
        q = f3['hcratio'].quantile(0.99)
        # get the 90% quantile of Baseline HC
        q2 = f3['Baseline HC'].quantile(0.99)

        # remove outliers
        f3 = f3[(f3['hcratio'] < q) & (f3['Baseline HC'] < q2)]
        sns.scatterplot(ax=ax[r][c], y='additional', x='Baseline HC', data=f3, s=4, palette=mypalette, c='blue')
        # set only the color of the circles to what i want which is orange

 

        # plot a vertical line at x = q
        # plt.axvline(x=q, color='red')
        # # plot a horizontal line at y = q2
        # plt.axhline(y=q2, color='red')

        # # limit x, y axes by quartiles
        # plt.xlim(1.0, q)
        # plt.ylim(30000, q2)

        # # limit x by max hcratio
        # ax[r][c].set_ylim(f3['hcratio'].min()-0.1, f3['hcratio'].max() + 0.1)
        # # limit y thhe same
        # ax[r][c].set_xlim(f3['Baseline HC'].min() - 5000, f3['Baseline HC'].max() + 5000)

        ax[r][c].set_ylim(0, 250000)
        ax[r][c].set_xlim(40000, 220000)

        # remove x axis label
        ax[r][c].set_xlabel('')
        # remove y axis label
        ax[r][c].set_ylabel('')

        # draw grid
        ax[r][c].grid(True, which='both', axis='both', linestyle='--', color='black', alpha=0.25)

        # fit a line to hcratio (x) and Baseline HC (y)
        # get the slope and intercept
        slope, intercept, r_value, p_value, std_err = stats.linregress(f3['Baseline HC'], f3['additional'])
        ax[r][c].text(90000, 210000, 'pearson correlation coefficient = %0.2f' % (r_value), color='blue', fontsize=8)

        # fit an third order curve to hcratio (x) and Baseline HC (y)
        # get the coefficients
        coefficients, residuals, rank, singular_values, rcond = np.polyfit(f3['Baseline HC'], f3['additional'], 3, full=True)
        # get the polynomial
        polynomial = np.poly1d(coefficients)
        # get the x values
        x_axis = sorted(f3['Baseline HC'].values)
        # plot the polynomial
        ax[r][c].plot(x_axis, polynomial(x_axis), 'black', label='Polynomial fit', linewidth=2)
        ax[r][c].plot(x_axis, polynomial(x_axis), 'orange', label='Polynomial fit', linewidth=1)

        # import ticker

        # shorten y-axis labels by adding k
        ax[r][c].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'k'))

        # shorten x-axis labels by adding k
        ax[r][c].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'k'))

        ax[r][c].text(185000, 150000, 'Chip ' + str(chip_id), color='black', bbox=dict(facecolor='white', edgecolor='black', pad=10.0, boxstyle='round,pad=0.5',alpha=0.1), fontsize=10)

        # remove legend from ax
        ax[r][c].legend().remove()

# put text in the middle
fig.text(0.5, 0.04, 'Hammer count to induce the first bitflip $(HC_{first})$', ha='center', va='center')

# put text as y axis in the middle
fig.text(0.055, 0.5, 'Additional (over $HC_{first}$) hammer count to induce the $10^{th}$ bitflip', ha='center', va='center', rotation='vertical')

# reduce vertical space between subplots
plt.subplots_adjust(hspace=0.2)
# reduce horizontal space between subplots
plt.subplots_adjust(wspace=0.17)

# y axis is $HC_{first}$
# plt.xlabel('Hammer count to induce the first bitflip $(HC_{first})$')
# # x axis is Normalized hammer count to induce the $10^{th}$ bitflip
# plt.ylabel('Hammer count to induce\nthe $10^{th}$ bitflip (normalized to $HC_{first}$)')

# save the plot
plt.savefig('figure11.pdf', bbox_inches='tight')

### Create figure 12 (this takes ~40 minutes)

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[2], ext_colors[4], ext_colors[6], ext_colors[7], ext_colors[5], ext_colors[3], ext_colors[1]]

mypalette = sns.color_palette(ext_colors)

mypalette


# summarize per bit data into per row
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

host_machines = [
  'safari-fpga37',
  'safari-fpga55',
  'safari-fpga56',
  'safari-fpga57',
  'safari-fpga58',
  'safari-fpga59'
]

rows_tested_for_bitflips = 384

ras_scale = [0, 1, 2, 4, 8, 16, 32, 64, 134.48, 1210.34]

ras_to_microsec = [0.0333 + (x*33.333/1000) for x in ras_scale]

print(ras_to_microsec)

dfs = []

# Loop over all CSV files in the directory
for host_machine in host_machines:
  dir_path = './' + 'fast_forward_data/' + host_machine
  for filename in os.listdir(dir_path):
    if 'aggon' in filename and 'rowpress' in filename:
      print(filename)
      tokens = filename.split('_')
      ch_num = int(tokens[1].split('-')[1])
      pc_num = int(tokens[2].split('-')[1])
      bank_num = int(tokens[3].split('-')[1])
      pattern = "AA"
      taggon = int(tokens[5].split('-')[1].split('.')[0])
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename)
      # add the channel, pc, bank and pattern numhcfirsts as columns
      df['machine'] = host_machine
      df['channel'] = ch_num

      if taggon > 7:
        rettime = "10530"
        if taggon == 8:
          rettime = "1170"
        retention_file = "retention_ch-"+str(ch_num)+"_pch-"+str(pc_num)+"_bank-"+str(bank_num)+"_dp-0xAA_ms-" + rettime + ".csv"
        print(retention_file)
        retention_df = pd.read_csv(dir_path + '/' + retention_file)

        retention_df['machine'] = host_machine
        retention_df['channel'] = ch_num
        # machine, channel, pseudo_channel, bank, row, column, bit indicate position of a bitflip in both dataframes
        # remove from df all rows that are in retention_df
        df = df[~df[['machine', 'channel', 'pseudo_channel', 'bank', 'row', 'column', 'bit']].apply(tuple,1).isin(retention_df[['machine', 'channel', 'pseudo_channel', 'bank', 'row', 'column', 'bit']].apply(tuple,1))]

      # group by machine, channel, pseudo_channel, bank, row, pattern, taggon
      # count the entries and add the count as new column
      df = df.groupby(['machine', 'channel', 'pseudo_channel', 'bank', 'row', 'data_pattern', 'aggon']).size().reset_index(name='count')
      dfs.append(df)
      # append the dataframe to the list

# # Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# df = pd.read_csv('rowpress_bitflips_row_summary.csv')

# map the channels by decrementing them by 8
df['CH'] = df['channel']
df['CH'] = df['CH'].map(lambda x: x - 8)

# rename machine using the following map
machine_map = {
  'safari-fpga37': 'Chip 0',
  'safari-fpga55': 'Chip 1',
  'safari-fpga56': 'Chip 2',
  'safari-fpga57': 'Chip 3',
  'safari-fpga58': 'Chip 4',
  'safari-fpga59': 'Chip 5'
}

df['machine'] = df['machine'].map(machine_map)

pattern_map = {
  '170': 'AA'
}

# modify pattern column only if the pattern is not one of the values in the map
df['pattern'] = df['data_pattern'].map(lambda x: pattern_map[x] if x in pattern_map else x)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

# rename row_errors to bitflips
df.rename(columns={'count': 'bitflips'}, inplace=True)

df['bitflips'] = df['bitflips'] / 5

# create bit error rate column 
# ber = bitflips / row size (1024*8)
df['ber'] = df['bitflips'] / (1024*8) * 100

sidebyside = True

hcf_chip_ch = df.copy()

# flatten the multi-index
hcf_chip_ch.reset_index(inplace=True)

# select only chip, channel and bitflips
hcf_chip_ch = hcf_chip_ch[['chip', 'CH', 'aggon', 'ber']]

# create a subplot for each chip
if sidebyside:
  fig, axes = plt.subplots(nrows=len(hcf_chip_ch['chip'].unique()),ncols=2, figsize=(8.1, 1.5*len(hcf_chip_ch['chip'].unique())))
else:
  fig, axes = plt.subplots(nrows=len(hcf_chip_ch['chip'].unique()), figsize=(8.1, 2.0*len(hcf_chip_ch['chip'].unique())))

# select the taggon values at 0, 9, 11, 19
# hcf_chip_ch = hcf_chip_ch[hcf_chip_ch['taggon'].isin([0, 1, 2, 3])]
# ras_to_microsec = ['29.0ns', '58.0ns', '87.0ns', '116.0ns']

# mapping from ras_scale to ras_to_microsec is 29.0 + (x*29.0) for x in ras_scale
# create ras_to_microsec list for each element in ras_scale
ras_scale = [0, 1, 2, 3, 8, 16, 32, 64, 134.48, 1210.34]
ras_to_microsec = [0.0333 + (x*33.333/1000) for x in ras_scale]

# ras_scale = ras_scale[:4]
ras_to_microsec = [29.0 + (x*29.0) for x in ras_scale]

# format ras_to_microsec to have 1 decimal place
ras_to_microsec = [round(x, 1) for x in ras_to_microsec]

# format ras_to_microsec as a string with ns appended; if an element is larger than 1000, convert it to us
ras_to_microsec = [str(x) + 'ns' if x < 1000 else str(round(x/1000,1)) + 'us' for x in ras_to_microsec]

ras_to_microsec[8:] = ["$t_{REFI}$ (3.9$\mu{}s$)", "$9*t_{REFI}$ (35.1$\mu{}s$)"]

ras_to_microsec[0] = "$t_{RAS}$\n(29.0ns)"

hcf_chip_ch['aggon_verbose'] = hcf_chip_ch['aggon'].apply(lambda x: ras_to_microsec[x])

# remove aggon larger than 4 from hcf_chip_ch
# hcf_chip_ch = hcf_chip_ch[hcf_chip_ch['aggon'] < 4]

# create a bar plot for each chip
if len(hcf_chip_ch['chip'].unique()) == 1:
  sns.boxplot(x='aggon', y='ber', data=hcf_chip_ch, ax=axes, showfliers=False, hue="CH", palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"5"})
  axes.text(0.1, 0.8, hcf_chip_ch['chip'].unique(), horizontalalignment='center', verticalalignment='center', transform=axes.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
  axes.get_legend().remove()
  axes.set_xlabel('Aggressor row on time (tAggON)')
  axes.set_xticklabels(ras_to_microsec[:10])
  axes.set_ylabel('')

else:
  for chip, ax in zip(hcf_chip_ch['chip'].unique(), axes):
    if sidebyside:
      sns.boxplot(x='aggon', y='ber', data=hcf_chip_ch[(hcf_chip_ch['chip'] == chip) & (hcf_chip_ch['aggon'] < 4)], ax=ax[0], showfliers=False, hue="CH", palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"5"})
      sns.boxplot(x='aggon', y='ber', data=hcf_chip_ch[(hcf_chip_ch['chip'] == chip) & (hcf_chip_ch['aggon'] > 7)], ax=ax[1], showfliers=False, hue="CH", palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"5"})
    else:
      sns.boxplot(x='aggon', y='ber', data=hcf_chip_ch[hcf_chip_ch['chip'] == chip], ax=ax, showfliers=False, hue="CH", palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"5"})
    
    if sidebyside:
      ax[0].text(0.15, 0.8, chip, horizontalalignment='center', verticalalignment='center', transform=ax[0].transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
      # ax[1].text(0.1, 0.8, chip, horizontalalignment='center', verticalalignment='center', transform=ax[1].transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
      ax[0].get_legend().remove()
      ax[1].get_legend().remove()
      if chip != 'Chip 5':
        ax[0].set_xlabel('')
        ax[1].set_xlabel('')
        ax[0].set_xticklabels([])
        ax[1].set_xticklabels([])
        ax[0].set_xticks([])
        ax[1].set_xticks([])

      if chip == 'Chip 0':
        # prepend 'CH' to legend labels
        handles, labels = ax[0].get_legend_handles_labels()
        ax[0].legend(handles, ['CH' + label for label in labels], loc='upper center', title='Channel', ncol=8, bbox_to_anchor=(1.05, 1.55), fancybox=True, shadow=True, columnspacing=0.5)
        # handles, labels = ax[1].get_legend_handles_labels()
        # ax[1].legend(handles, ['CH' + label for label in labels], loc='upper center', title='Channel', ncol=8, bbox_to_anchor=(0.5, 1.5), fancybox=True, shadow=True, columnspacing=0.5)
      if chip == 'Chip 5':
        # ax[0].set_xlabel('Aggressor row on time (tAggON)')
        # ax[1].set_xlabel('Aggressor row on time (tAggON)')
        ax[0].set_xlabel('')
        ax[1].set_xlabel('')
        # format ras_to_microsec to string with two decimal places
        # ras_to_microsecstr = [float("%.2f" % (t)) for t in ras_to_microsec]
        # set the x-axis labels to the formatted values
        # ax.set_xticklabels([str(float(t)) for t in ras_to_microsecstr])
        ax[0].set_xticklabels(ras_to_microsec[:4])
        ax[1].set_xticklabels(ras_to_microsec[8:])
        ax[0].text(1.1, -0.40, "Aggressor row on time ($t_{AggON}$)", horizontalalignment='center', verticalalignment='center', transform=ax[0].transAxes, fontsize=12)

        # rotate x-axis labels
        # for tick in ax.get_xticklabels():
        #   tick.set_rotation(90)


      # ax.set_yticks([x * 40000 for x in range(7)])
      # ax.set_yticklabels([str(int(int(t) / 1000))+"K" if t > 0 else "0" for t in ax.get_yticks()])
      ax[0].set_ylabel('')
      ax[1].set_ylabel('')

      # make x width more narrow
      # ax.set_xlim(-0.5, 3.5)

      # Put grid lines in the background
      ax[0].set_axisbelow(True)
      ax[1].set_axisbelow(True)
      ax[0].grid(color='gray', linestyle='dashed')
      ax[1].grid(color='gray', linestyle='dashed')
      # Put grid lines only on y-axis
      ax[0].xaxis.grid(False)
      ax[1].xaxis.grid(False)
      # ax.set_yscale('log')
      # ax.set_yticks([1, 10, 100, 1000, 10000, 100000])


      ax[0].set_ylim(0, 2.8)
      ax[1].set_ylim(0, 70)
      # ax.set_yticks([0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8])
      ax[0].yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
      ax[1].yaxis.set_major_formatter(FormatStrFormatter('%.0f'))

    else:
      ax.text(0.1, 0.8, chip, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
      ax.get_legend().remove()
      if chip != 'Chip 5':
        ax.set_xlabel('')
        ax.set_xticklabels([])
        ax.set_xticks([])

      if chip == 'Chip 1':
        # prepend 'CH' to legend labels
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, ['CH' + label for label in labels], loc='upper center', title='Channel', ncol=8, bbox_to_anchor=(0.5, 1.5), fancybox=True, shadow=True, columnspacing=0.5)
      if chip == 'Chip 5':
        ax.set_xlabel('Aggressor row on time (tAggON)')
        # format ras_to_microsec to string with two decimal places
        # ras_to_microsecstr = [float("%.2f" % (t)) for t in ras_to_microsec]
        # set the x-axis labels to the formatted values
        # ax.set_xticklabels([str(float(t)) for t in ras_to_microsecstr])
        ax.set_xticklabels(ras_to_microsec)

        # rotate x-axis labels
        # for tick in ax.get_xticklabels():
        #   tick.set_rotation(90)


      # ax.set_yticks([x * 40000 for x in range(7)])
      # ax.set_yticklabels([str(int(int(t) / 1000))+"K" if t > 0 else "0" for t in ax.get_yticks()])
      ax.set_ylabel('')

      # make x width more narrow
      # ax.set_xlim(-0.5, 3.5)

      # Put grid lines in the background
      ax.set_axisbelow(True)
      ax.grid(color='gray', linestyle='dashed')
      # Put grid lines only on y-axis
      ax.xaxis.grid(False)
      # ax.set_yscale('log')
      # ax.set_yticks([1, 10, 100, 1000, 10000, 100000])


      # ax.set_ylim(0, 2.8)
      # ax.set_yticks([0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8])
      ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))

      # ax.axvspan(boundaries[8]/sample_size, boundaries[9]/sample_size - 1, facecolor=(1,0.2,0.2), alpha=0.2)
      # draw a vspan between boxes at the 3.9us 
      # ax.axvspan(0.5, 1.5, facecolor=(1,0.5,0.2), alpha=0.1)
      # # also add lines
      # ax.axvline(0.5, color='black', linestyle='--')
      # ax.axvline(1.5, color='black', linestyle='--')
      # # draw a vspan between boxes at the 35.1us 
      # ax.axvspan(1.5, 2.5, facecolor=(1,0.2,0.2), alpha=0.1)
      # # also add lines
      # ax.axvline(2.5, color='black', linestyle='--')

      # add text to the first vspan saying "tREFI"
      # ax.text(0.38, 0.2, '$t_{REFI}$', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='orange', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
      # ax.text(0.63, 0.2, '$9*t_{REFI}$', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='red', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)


fig.subplots_adjust(hspace=0.1,wspace=0.13)
    

# add text as y axis label that spans all subplots
fig.text(0.065, 0.5, 'RowHammer Bit Error Rate ($BER$)', va='center', rotation='vertical', fontsize=12)

# save figure
fig.savefig('figure12.pdf', bbox_inches='tight')
# average ber at each taggon
print(hcf_chip_ch.groupby(['aggon_verbose'])['ber'].mean())

### Create figure 13

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[6], ext_colors[4]]

mypalette = sns.color_palette(ext_colors)

mypalette
 
# read all csv files in this folder and concatenate them into one dataframe
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

host_machines = [
  'safari-fpga37',
  'safari-fpga55',
  'safari-fpga56',
  'safari-fpga57',
  'safari-fpga58',
  'safari-fpga59'
]

rows_tested_for_bitflips = 384

ras_scale = [0, 1, 2, 4, 8, 16, 32, 64, 118, 256, 512, 1063, 5000, 10000, 20000, 50000, 100000, 200000, 500000, 1000000]

ras_to_microsec = [0.0333 + (x*33.333/1000) for x in ras_scale]

print(ras_to_microsec)

dfs = []

# Loop over all CSV files in the directory
for host_machine in host_machines:
  dir_path = './' + 'fast_forward_data/' + host_machine
  for filename in os.listdir(dir_path):
    if 'AggOn' in filename and 'hcfirst' in filename:
      tokens = filename.split('_')
      ch_num = int(tokens[2])
      pc_num = int(tokens[4])
      bank_num = int(tokens[6])
      pattern = tokens[8]
      taggon = int(tokens[10].split('.')[0])
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename, header=None, names=['hcfirst'])
      # add the channel, pc, bank and pattern numhcfirsts as columns
      df['CH'] = ch_num
      df['PC'] = pc_num
      df['bank'] = bank_num
      df['pattern'] = pattern
      df['machine'] = host_machine
      df['taggon'] = taggon
      df['row'] = df.index
      # append the dataframe to the list
      dfs.append(df)

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# map the channels by decrementing them by 8
df['CH'] = df['CH'].map(lambda x: x - 8)
 
# remove channels greater than 3
df = df[df['CH'] < 3]

def map_row(row):
  if row < 128:
    return row
  elif row < 256:
    return row + 7936
  else:
    return row + 15742

# apply map_row to the row column
df['row'] = df['row'].map(map_row)

# rename machine using the following map
machine_map = {
  'safari-fpga37': 'Chip 0',
  'safari-fpga55': 'Chip 1',
  'safari-fpga56': 'Chip 2',
  'safari-fpga57': 'Chip 3',
  'safari-fpga58': 'Chip 4',
  'safari-fpga59': 'Chip 5'
}
df['machine'] = df['machine'].map(machine_map)

pattern_map = {
  '0': '00',
  '1': 'FF',
  '2': '55',
  '3': 'AA'
}

# modify pattern column only if the pattern is not one of the values in the map
df['pattern'] = df['pattern'].map(lambda x: pattern_map[x] if x in pattern_map else x)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

# a new data point for each row is created for a new worst case data pattern
# this is the data pattern that maximizes hcfirst for a row in a given channel, pc, bank, chip, with different patterns

# remove rows with hcfirst >= 256000
df = df[df['hcfirst'] < 256000]

# a row should have an entry for each taggon
# if a row does not have an entry for a pattern, then remove it completely
# remove rows that do not have all patterns
df = df.groupby(['chip', 'CH', 'PC', 'row', 'bank', 'pattern']).filter(lambda x: len(x['taggon'].unique()) == 20)

# add new column ras_microsec mapping from taggon to ras_to_microsec
df['ras_microsec'] = df['taggon'].map(lambda x: ras_to_microsec[x])

# add new column experiment_time as the product of ras_to_microsec and hcfirst and 2
df['experiment_time'] = df['hcfirst'] * 2 * df['ras_microsec']

df_large = df[df['experiment_time'] >= 34000]

# if experiemtn times are greater than 32000, remove row
df = df[df['experiment_time'] < 34000]

df = df[df['taggon'].isin([0, 9, 11, 18])]

df = df.groupby(['chip', 'CH', 'PC', 'row', 'bank', 'pattern']).filter(lambda x: len(x['taggon'].unique()) == 4)

df2 = df.copy()

df2 = df2[df2['taggon'].isin([0, 9, 11, 18])]

# remove rows that do not exist for all taggons
df2 = df2.groupby(['chip', 'CH', 'PC', 'row', 'bank', 'pattern']).filter(lambda x: len(x['taggon'].unique()) == 4)

df2

hcf_chip_ch = df.copy()
# flatten the multi-index
hcf_chip_ch.reset_index(inplace=True)

# select only chip, channel and bitflips
hcf_chip_ch = hcf_chip_ch[['chip', 'CH', 'taggon', 'hcfirst']]

# create a subplot for each chip
fig, axes = plt.subplots(nrows=len(hcf_chip_ch['chip'].unique()), figsize=(8.1, 2.0*len(hcf_chip_ch['chip'].unique())))

# select the taggon values at 0, 9, 11, 19
hcf_chip_ch = hcf_chip_ch[hcf_chip_ch['taggon'].isin([0, 9, 11, 18])]
ras_to_microsec = ['29.0ns', '3.9$\mu$s', '35.1$\mu$s', '16.0ms']

sample_size = [153, 649, 225, 56, 145, 62]

# create a bar plot for each chip
for chip, ax, i in zip(hcf_chip_ch['chip'].unique(), axes, range(len(hcf_chip_ch['chip'].unique()))):
  sns.boxplot(x='taggon', y='hcfirst', data=hcf_chip_ch[hcf_chip_ch['chip'] == chip], ax=ax, showfliers=False, hue="CH", palette=mypalette, whis=(0,100), showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black","markersize":"5"})
  ax.text(0.12, 0.35, chip, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
  ax.get_legend().remove()
  if chip != 'Chip 5':
    ax.set_xlabel('')
    ax.set_xticklabels([])
    ax.set_xticks([])

  if chip == 'Chip 0':
    # prepend 'CH' to legend labels
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['CH' + label for label in labels], loc='upper center', title='Channel', ncol=8, bbox_to_anchor=(0.5, 1.5), fancybox=True, shadow=True, columnspacing=0.5)
  if chip == 'Chip 5':
    ax.set_xlabel('Aggressor row on time (tAggON)')
    # format ras_to_microsec to string with two decimal places
    # ras_to_microsecstr = [float("%.2f" % (t)) for t in ras_to_microsec]
    # set the x-axis labels to the formatted values
    # ax.set_xticklabels([str(float(t)) for t in ras_to_microsecstr])
    ax.set_xticklabels(ras_to_microsec)

    # rotate x-axis labels
    # for tick in ax.get_xticklabels():
    #   tick.set_rotation(90)


  # ax.set_yticks([x * 40000 for x in range(7)])
  # ax.set_yticklabels([str(int(int(t) / 1000))+"K" if t > 0 else "0" for t in ax.get_yticks()])
  ax.set_ylabel('')

  # make x width more narrow
  ax.set_xlim(-0.5, 3.5)

  # Put grid lines in the background
  ax.set_axisbelow(True)
  ax.grid(color='gray', linestyle='dashed')
  # Put grid lines only on y-axis
  ax.xaxis.grid(False)
  ax.set_yscale('log')
  ax.set_yticks([1, 10, 100, 1000, 10000, 100000])

  # ax.axvspan(boundaries[8]/sample_size, boundaries[9]/sample_size - 1, facecolor=(1,0.2,0.2), alpha=0.2)
  # draw a vspan between boxes at the 3.9us 
  ax.axvspan(0.5, 1.5, facecolor=(1,0.5,0.2), alpha=0.1)
  # also add lines
  ax.axvline(0.5, color='black', linestyle='--')
  ax.axvline(1.5, color='black', linestyle='--')
  # draw a vspan between boxes at the 35.1us 
  ax.axvspan(1.5, 2.5, facecolor=(1,0.2,0.2), alpha=0.1)
  # also add lines
  ax.axvline(2.5, color='black', linestyle='--')

  # add text to the first vspan saying "tREFI"
  ax.text(0.38, 0.2, '$t_{REFI}$', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='orange', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)
  ax.text(0.63, 0.2, '$9*t_{REFI}$', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='red', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)

  # add text to signify sample size for each subplot
  ax.text(0.88, 0.5, str(sample_size[i]) + ' rows', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='grey', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.5), fontsize=12)

fig.subplots_adjust(hspace=0.15)
    

# add text as y axis label that spans all subplots
fig.text(0.04, 0.5, 'Minimum Hammer Count to Induce the First Bitflip ($HC_{first}$)', va='center', rotation='vertical', fontsize=12)

# save figure
fig.savefig('figure13.pdf', bbox_inches='tight')

# average HCfirst across all chips for each taggon value
print(hcf_chip_ch.groupby(['taggon'])['hcfirst'].mean())
print(hcf_chip_ch.groupby(['taggon'])['hcfirst'].min())



### Create figure 14

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

def read_data(filename):
    file = open(filename, "r")
    t_df = pd.DataFrame(columns=["pivot_row", "bitflips"])
    for line in file.readlines():
        pivot_row = int(line.partition(":")[0].strip())
        bitflips = [int(bfs) for bfs in line.partition(":")[2].strip().split(" ")]
        bitflips = sum(bitflips)
        t_df.loc[len(t_df)] = [pivot_row, bitflips]
    return t_df
ph = [plt.plot([],marker="", ls="")[0]]


num_dummy_rows_list = [4, 5, 6, 7, 8, 9, 10]
hammers_per_ref_loop_list = [
    "10 10",
    "12 12",
    "14 14",
    "16 16",
    "18 18",
    "20 20",
    "22 22",
    "24 24",
    "26 26",
    "28 28",
    "30 30",
    "32 32",
    "34 34",
    "36 36",
    "38 38",
]


before = True
after = False
trrref_sync = False

df = pd.DataFrame()
for num_dummy_rows in num_dummy_rows_list:
    for hammers_per_ref_loop in hammers_per_ref_loop_list:
        for shift_refs in [7]:
            # out_filename = "f" + str(shift) + ".txt"
            out_filename = "fast_forward_data/utrr_results/" + \
                    "dummy_" + str(num_dummy_rows) + \
                    "-h_" + hammers_per_ref_loop.replace(" ", "_") + \
                    "-before_" + str(before) + \
                    "-after_" + str(after) + \
                    "-trrrefsync_" + str(trrref_sync) + \
                    ".txt"
            if(not os.path.exists(out_filename)):
                continue

            t_df = read_data(out_filename)
            t_df["num_dummy_rows"] = num_dummy_rows
            t_df["hammers_per_ref_loop"] = hammers_per_ref_loop.split(" ")[0]

            df = pd.concat([df, t_df], ignore_index=True)
            
# for hammers in hammers_per_ref_loop_list:
#     df.loc[len(df)] = [-1, 0, 3, hammers.split(" ")[0]]

df["num_dummy_rows"] = df["num_dummy_rows"].astype(int)
df["hammers_per_ref_loop"] = df["hammers_per_ref_loop"].astype(int)

df.loc[len(df)] = [-1, 0, 4, 38]
df.loc[len(df)] = [-1, 0, 5, 38]
df.loc[len(df)] = [-1, 0, 6, 38]
df.loc[len(df)] = [-1, 0, 7, 38]
df.loc[len(df)] = [-1, 0, 8, 38]
df.loc[len(df)] = [-1, 0, 9, 38]
df.loc[len(df)] = [-1, 0, 10, 38]
df.loc[len(df)] = [-1, 0, 5, 36]

df.sort_values(by=['hammers_per_ref_loop', 'num_dummy_rows'], inplace=True)

t_df = df.copy()

t_df = t_df[t_df['hammers_per_ref_loop'] <= 34]

# new column ber is bitflips per row divided by 16*1024*8 then multiplied by 100
t_df['ber'] = (t_df['bitflips'] / (8*1024*8)) * 100

fig, ax = plt.subplots(figsize=(6, 3))
# plt.title("Average bitflips per row", )
sns.boxplot(x="num_dummy_rows", y="ber", hue="hammers_per_ref_loop", whis=[0,100], data=t_df, palette="pastel", zorder=10, width=0.8, 
            ax=ax, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"3"})

# set legend title "Aggressor row activation count"

ax.legend(fontsize=9, loc="upper center", bbox_to_anchor=(0.5, 1.25), title='Aggressor row activation count in a $t_{REFI}$', title_fontsize=12, ncols=9, columnspacing=0.5, fancybox=True)

ax.set_ylabel("RowHammer Bit Error Rate ($BER$)", fontsize=12)
ax.set_xlabel("Number of dummy rows", fontsize=12)
ax.tick_params(axis="x", direction="out", labelsize=12, color='black')
ax.tick_params(axis="y", direction="out", labelsize=12, color='black')


plt.grid(axis='y', alpha=0.8, zorder=0)

plt.savefig('./figure14.png', bbox_inches='tight', pad_inches=0.01)
plt.savefig('./figure14.pdf', bbox_inches='tight', pad_inches=0.01)

### Create figure 15

In [ ]:
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
import numpy as np

# giray's style
scale_factor = 16

colors = sns.color_palette("plasma", 4 * scale_factor)

ext_colors = []
i = 0 
while i < 4:
    c = colors[i*scale_factor]
    cn = tuple([cc * 0.9 for cc in c])
    ext_colors.append(cn)
    ext_colors.append(c)
    i+=1

print(ext_colors)

ext_colors = [ext_colors[0], ext_colors[2], ext_colors[4], ext_colors[6], ext_colors[7], ext_colors[5], ext_colors[3], ext_colors[1]]

mypalette = sns.color_palette(ext_colors)

mypalette

########################################################
######## Run once to generate the summary ##############
########################################################

# read all csv files in this folder and concatenate them into one dataframe
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

host_machines = [
  'safari-fpga58'
]

rows_tested_for_bitflips = 3072

dfs = []

# Loop over all CSV files in the directory
for host_machine in host_machines:
  dir_path = './' + 'fast_forward_data/' + host_machine
  for filename in os.listdir(dir_path):
    if filename.startswith('rowhammer') and 'tAggOn' not in filename:
      tokens = filename.split('_')
      ch_num = int(tokens[2])
      pc_num = int(tokens[4])
      bank_num = int(tokens[6])
      pattern = tokens[8].split('.')[0]
      # create a dataframe from the csv file
      df = pd.read_csv(dir_path + '/' + filename)
      # add the channel, pc, bank and pattern numbers as columns
      df['CH'] = ch_num
      df['PC'] = pc_num
      df['bank'] = bank_num
      df['pattern'] = pattern
      df['machine'] = host_machine
      # if an entry in the df has byte_errors = 0, remove it
      df = df[df['byte_errors'] != 0] 
      # append the dataframe to the list
      dfs.append(df)

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# map the channels by decrementing them by 8
df['CH'] = df['CH'].map(lambda x: x - 8)

# map the rows in the following ranges to the following ranges
# [0-1023] -> [0-1023]
# [1024-2047] -> [7168-8191]
# [2048-3071] -> [13310-14333]

def map_row(row):
  if row < 1024:
    return row
  elif row < 2048:
    return row + 6144
  else:
    return row + 11262

common_rows = [
  range(0, 1024, 1),
  range(7168, 8192, 1),
  range(13310, 14334, 1)
]

common_banks = [
  0, 1, 2
]

# group by machine, channel, pc, bank, pattern
# if the group has 3072 rows, then apply map row
# otherwise, leave the row as is
df['row'] = df.groupby(['machine', 'CH', 'PC', 'bank', 'pattern'])['row'].transform(lambda x: x.map(map_row) if len(x) == rows_tested_for_bitflips else x)

# rename machine using the following map
machine_map = {
  'safari-fpga58': 'Chip 4'
}
df['machine'] = df['machine'].map(machine_map)

pattern_map = {
  '0': '00',
  '1': 'FF',
  '2': '55',
  '3': 'AA'
}

# modify pattern column only if the pattern is not one of the values in the map
df['pattern'] = df['pattern'].map(lambda x: pattern_map[x] if x in pattern_map else x)

# rename machine column to chip
df.rename(columns={'machine': 'chip'}, inplace=True)

# rename row_errors to bitflips
df.rename(columns={'byte_errors': 'bitflips'}, inplace=True)

df['bitflips'] = df['bitflips'] / 5

# create bit error rate column 
# ber = bitflips / row size (1024*8)
# df['ber'] = df['bitflips'] / (1024*8) * 100

# filter out uncommon rows
df_common_rows = df[df['row'].isin(common_rows[0]) | df['row'].isin(common_rows[1]) | df['row'].isin(common_rows[2])].copy()
df_common_banks_rows = df_common_rows[df_common_rows['bank'].isin(common_banks)].copy()

# a new data point for each row is created for a new worst case data pattern
# this is the data pattern that maximizes ber for a row in a given channel, pc, bank, chip, with different patterns

# group by chip, channel, pc, bank, row
# for each group, find the pattern that maximizes ber
# create a new dataframe with the following columns: chip, channel, pc, bank, row, pattern, ber
# df_worst_case = df_common_banks_rows.groupby(['chip', 'CH', 'PC', 'bank', 'row'])[['pattern', 'ber']].max().reset_index()

# df_worst_case.reset_index(drop=True, inplace=True)

# rename pattern values with "WCDP"
# df_worst_case['pattern'] = df_worst_case['pattern'].map(lambda x: 'WCDP')

# add df_worst_case to df
# df_common_banks_rows = pd.concat([df_common_banks_rows, df_worst_case], ignore_index=True)

df_common_banks_rows

# remove pattern
# df_common_banks_rows.drop(columns=['pattern'], inplace=True)

# round bitflips to the next largest integer
df_common_banks_rows['bitflips'] = df_common_banks_rows['bitflips'].map(lambda x: int(x) + 1)

# group by chip, CH, PC, bank, row, byte, reduce by summing
df_common_banks_rows = df_common_banks_rows.groupby(['chip', 'CH', 'PC', 'bank', 'row', 'pattern', 'byte']).sum().reset_index()

# create new column word = byte/8 (integer division)
df_common_banks_rows['word'] = df_common_banks_rows['byte'].map(lambda x: int(x / 8))


df_errs_per_word = df_common_banks_rows.groupby(['CH','PC','bank','row','pattern','word']).sum().reset_index()
df_errs_per_word.drop(columns=['byte'], inplace=True)
# sort by decreasing number of bitflips
df_errs_per_word.sort_values(by=['bitflips'], ascending=False, inplace=True)

# order for the pattern column is:
pattern_order = ['FF', '00', 'AA', '55', 'WCDP']
# apply this order
df_errs_per_word['pattern'] = pd.Categorical(df_errs_per_word['pattern'], pattern_order)
# sort by pattern
df_errs_per_word.sort_values(by=['pattern'], inplace=True)

data_patterns = {
  'FF': "Rowstripe0",
  '00': "Rowstripe1",
  'AA': "Checkered0",
  '55': "Checkered1"
}

df_errs_per_word['pattern'] = df_errs_per_word['pattern'].map(data_patterns)
# categorize the entries based on the number of bitflips:

# more than 2 bitflips, SECDED undetectable
# more than 1 bitflips, SECDED uncorrectable
# others, SECDED correctable

df_errs_per_word["ECC Category"] = pd.cut(df_errs_per_word['bitflips'], bins=[0, 1, 2, np.inf], labels = ['correctable', 'uncorrectable', 'undetectable'])
df_errs_per_word["Number of Bitflips"] = pd.cut(df_errs_per_word['bitflips'], bins=[0, 1, 2, np.inf], labels = ['<=1', '<=2', '>2'])
# df_errs_per_word.drop(columns=['Unnamed: 0'], inplace=True)
df_errs_per_word

ax = plt.gca()
ax.yaxis.grid(color='gray', linestyle='dashed')

sns.countplot(x='Number of Bitflips', hue='pattern', data=df_errs_per_word, palette=mypalette, edgecolor='black', linewidth=1.5)

# make figure size smaller
plt.gcf().set_size_inches(7, 3)

# rename y axis ticks to the K and M format (1000 -> 1K, 1000000 -> 1M)
import matplotlib.ticker as mtick
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
# rename to M format only (divide by one million and append 'M')
# ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
ax.set_yticklabels([0] + [str(x/1000000)+'M' for x in ax.get_yticks()[1:]])

# increase al font sizes
ax.tick_params(axis='both', which='major', labelsize=12)

# rename legend title to Data pattern
ax.legend(title='Data pattern', fontsize=12, fancybox=True, shadow=True, title_fontsize=13, ncols=1, loc = 'center right', bbox_to_anchor=(1.35, 0.5))

# rename first x axis tick to <2, second to =2
ax.set_xticklabels(['1', '2', '>2'], fontsize=12, rotation=0)

# rename y axis label "Number of words (8 byte)"
ax.set_ylabel('Number of words (8 byte)\nthat exhibit bitflips', fontsize=14) 

ax.set_xlabel("Number of bitflips in a word", fontsize=14)

# save figure 'ecc_analysis.pdf'
plt.savefig('figure15.pdf', bbox_inches='tight')

# number of words with at least 1 bitflip and more than 2 bitflips for Checkered0
print('Number of words with at least 1 bitflip: ' + str(len(df_errs_per_word[(df_errs_per_word['pattern'] == 'Checkered0') & (df_errs_per_word['Number of Bitflips'] == '>2')])))
 